# QR Code Scanner with Local Data Handling

### 1. Importing Libraries

In [ ]:
import cv2
from pyttsx3 import init
import pandas as pd
from statistics import mode
from pyzbar.pyzbar import decode
import csv

### 2. Loading Local Data

In [ ]:
# Read the CSV file and specify the data types
df = pd.read_json('Products (1).json')
df = df.drop(['picture_url', 'brand_id', 'category_id'], axis=1)

# Initialize text-to-speech engine

In [ ]:
engine = init()
rate = engine.getProperty('rate')
engine.setProperty('rate', rate - 35)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

### 3. Defining Functions

In [ ]:
def speak(text):
    engine.say(text)
    engine.runAndWait()

def decode_qr_code(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    decoded_objects = decode(gray)
    if decoded_objects:
        return decoded_objects[0].data.decode('utf-8')
    else:
        return None

# ## 4. Capturing and Processing QR Codes

def capture_image():
    scanned_products = set()
    scanned_products_file = 'scanned_products.csv'
    detected_products = []
    selected_products = []
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            cv2.imshow('QR Code Scanner', frame)
            qr_data = decode_qr_code(frame)
            if qr_data:
                if qr_data not in selected_products:
                    if len(detected_products) == 10:
                        product_id = mode(detected_products)
                        result = df[df['code'] == product_id]
                        if not result.empty:
                            # Extract product information
                            product_name = result['product_name'].values[0]
                            description = result['description'].values[0]
                            price = result['price'].values[0]
                            currency = result['currency'].values[0]
                            # Construct product information text
                            product_text = f"Product Name: {product_name}\nDescription: {description}\nPrice: {price} {currency}"
                            # Speak the product information
                            speak(product_text)
                        selected_products.append(mode(detected_products))
                        detected_products = []
                        try:
                            with open(scanned_products_file, 'a', newline='') as csvfile:
                                writer = csv.writer(csvfile)
                                writer.writerow(product_text.split('\n'))
                        except IOError as e:
                            print(f"Error saving scanned product data: {e}")
                        scanned_products.add(qr_data)
                    else:
                        detected_products.append(qr_data)
                else:
                    continue
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

capture_image()